In [3]:
import numpy as np
import matplotlib.pyplot as plt
import wave
from scipy.linalg import toeplitz
from scipy.io.wavfile import write

In [4]:
def read_audio(path):
    signal = wave.open(path, 'rb')
    signal_data = signal.readframes(-1)
    signal_data = np.frombuffer(signal_data, dtype='int16')
    return signal_data, signal.getframerate()

In [36]:
def prep(signal, P):
    N = len(signal)
    corr = np.correlate(signal, signal, mode='full')[N-1:]
    corr = corr/(N-P+1)
    corr_vec = corr[P:2*P]
    R = toeplitz(corr_vec)
    return R, corr_vec, corr

In [35]:
def train(Rx, corr_vec):
    wopt = np.linalg.inv(Rx) @ corr_vec
    return wopt

In [45]:
def process(signal, wopt):
    N = len(signal)
    y = np.zeros(N)
    for i in range(P, N):
        y[i] = wopt @ signal[i-P:i]
    return y

In [47]:
def export_audio(filename, signal, fs):
    audio = signal.astype('int16')
    write('./audio/'+filename+'.wav', fs, audio)
    print('Audio exported successfully!')

In [5]:
signal1, fs1 = read_audio('./audio/conv1.wav')
signal2, fs2 = read_audio('./audio/conv73.wav')

In [12]:
N = min(len(signal1), len(signal2))

In [14]:
signal1 = signal1[:N]
signal2 = signal2[:N]

In [15]:
signal = signal1 + signal2

In [1]:
P = 3

In [24]:
Rd1, corr_vec1, corr1 = prep(signal1, P)
Rd2, corr_vec2, corr2 = prep(signal2, P)
Rx, corr_vec, corr = prep(signal, P)

In [34]:
wopt1 = train(Rx, corr_vec1)
wopt2 = train(Rx, corr_vec2)

In [44]:
d1hat = process(signal, wopt1)
d2hat = process(signal, wopt2)

In [48]:
export_audio('d1hat', d1hat, fs1)
export_audio('d2hat', d2hat, fs2)